In [1]:
import pandas as pd
df = pd.read_csv('/home/vinay_dara/Desktop/HK6/MothersDay/Data/input/input_5_prepared_distinct.csv')
df['piece_item_desc'] = df['piece_item_desc'].astype(str)
df['PIECE_ITEM_DESC_TEST'] = df['piece_item_desc']
df['COMMODITY_DESC'] = df['piece_item_desc']

#df.head()

In [2]:
import re

#cleaning the text data
def pre_process(PIECE_ITEM_DESC_TEST):
    
    # lowercase
    PIECE_ITEM_DESC_TEST=PIECE_ITEM_DESC_TEST.lower()
    
    #remove tags
    PIECE_ITEM_DESC_TEST=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",PIECE_ITEM_DESC_TEST)
    
    # remove special characters and digits
    PIECE_ITEM_DESC_TEST=re.sub("(\\d|\\W)+"," ",PIECE_ITEM_DESC_TEST)
    
    return PIECE_ITEM_DESC_TEST

df['PIECE_ITEM_DESC_TEST'] = df['PIECE_ITEM_DESC_TEST'].apply(lambda x:pre_process(x))

#Lemmatization
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

def nltk2wn_tag(nltk_tag):
  if nltk_tag.startswith('J'):
    return wordnet.ADJ
  elif nltk_tag.startswith('V'):
    return wordnet.VERB
  elif nltk_tag.startswith('N'):
    return wordnet.NOUN
  elif nltk_tag.startswith('R'):
    return wordnet.ADV
  else:          
    return None

def lemmatize_sentence(sentence):
  nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
  wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
  res_words = []
  for word, tag in wn_tagged:
    if tag is None:            
      res_words.append(word)
    else:
      res_words.append(lemmatizer.lemmatize(word, tag))
  return " ".join(res_words)

df['PIECE_ITEM_DESC_TEST'] = df.PIECE_ITEM_DESC_TEST.apply(lemmatize_sentence)
#df.head()

In [78]:
#Text Mining and Representation
for idx, row in df.iterrows():    
    #flowers
    matchFlowers = re.search(r'\bflower\b|\bfresh flower\b|\bfoliage\b|\bcutflower\b|\bcutflowers\b|\bflowers\b',row['PIECE_ITEM_DESC_TEST'])
    matchTulips = re.search(r'\btulip\b|\btulips\b',row['PIECE_ITEM_DESC_TEST'])
    matchRoses = re.search(r'\brose\b|\broses\b',row['PIECE_ITEM_DESC_TEST'])
    matchLillies = re.search(r'\blily\b|\blillies\b',row['PIECE_ITEM_DESC_TEST'])
    matchTropical = re.search(r'\btropical* flower\b',row['PIECE_ITEM_DESC_TEST'])
    matchLoroco = re.search(r'\bloroco\b',row['PIECE_ITEM_DESC_TEST'])

    if matchFlowers:
        df.loc[idx,'COMMODITY_DESC'] = "CUT FLOWERS FOLIAGE CUTTIN"
    elif matchTulips:
        df.loc[idx,'COMMODITY_DESC'] = "TULIPS"
    elif matchRoses:
        df.loc[idx,'COMMODITY_DESC'] = "ROSES"
    elif matchLillies:
        df.loc[idx,'COMMODITY_DESC'] = "LILLIES"
    elif matchLoroco:
        df.loc[idx,'COMMODITY_DESC'] = "LOROCO"
    elif matchTropical:
        df.loc[idx,'COMMODITY_DESC'] = "TROPICAL FLOWERS"
    #elif True:
        #df.drop(idx, inplace=True)

In [79]:
df.to_csv('/home/vinay_dara/Desktop/HK6/MothersDay/Data/output/output_5.csv')